<a href="https://colab.research.google.com/github/jacksonStone/sentiment-analysis-with-hot-one-encoding-2/blob/main/DataGeneratorExperiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


First we need to generate a test data set.
I leverage Grok and the latest Llama 3 to do this.

In [ ]:
from groq import Groq
from google.colab import userdata
from time import sleep

client = Groq(
    api_key=userdata.get("GROQ_API_KEY"),
)
betterPositiveCompletions = []
for j in range(10):
  if(j > 0):
    sleep(50)
  for i in range(30):
    chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": "Generate 10 random positive statements. Be succient. One sentence per statement. Go straight to the content with no preamble. Structure them like so: STATEMENT:some positive statement.\nSTATEMENT:some other positive statement\netc.",
          }
      ],
      model="llama3-70b-8192",
    )
    betterPositiveCompletions.append(chat_completion.choices[0].message.content)
  print("finished batch: " + str(j) + " out of 100")
print(betterPositiveCompletions)
# persist chatCompletions to pickle
parsedPositiveCompletions = []
for i in betterPositiveCompletions:
  parsedPositiveCompletions += i.split("\nSTATEMENT:")[1:]
print(parsedPositiveCompletions)

betterNegativeCompletions = []
for j in range(10):
  if(j > 0):
    sleep(50)
  for i in range(30):
    chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": "Generate 10 random negative statements. Be succient. One sentence per statement. Go straight to the content with no preamble. Structure them like so: STATEMENT:some negative statement.\nSTATEMENT:some other negative statement\netc.",
          }
      ],
      model="llama3-70b-8192",
    )
    betterNegativeCompletions.append(chat_completion.choices[0].message.content)
  print("finished batch: " + str(j) + " out of 100")
print(betterNegativeCompletions)
# persist chatCompletions to pickle
parsedNegativeCompletions = []
for i in betterNegativeCompletions:
  parsedNegativeCompletions += i.split("\nSTATEMENT:")[1:]
print(parsedNegativeCompletions)

# persist chatCompletions to pickle
import pickle
with open('negativeChats2.pkl', 'wb') as f:
    pickle.dump(parsedNegativeCompletions, f)



KeyboardInterrupt: 

Then we save the data sets to Drive to prevent needing to rerun the 20min script

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
with open('/content/drive/MyDrive/negativeChats.pkl', 'wb') as f:
    pickle.dump(parsedNegativeCompletions, f)

In [ ]:
with open('/content/drive/MyDrive/positiveChats.pkl', 'wb') as f:
    pickle.dump(parsedPositiveCompletions, f)

Now I need to convert the data into numerical values

In [ ]:
import string
import pickle
from random import shuffle
positiveChats = []
negativeChats = []
with open('/content/drive/MyDrive/positiveChats.pkl', 'rb') as f:
    positiveChats = dict.fromkeys(pickle.load(f)).keys()
with open('/content/drive/MyDrive/negativeChats.pkl', 'rb') as f:
    negativeChats = dict.fromkeys(pickle.load(f)).keys()
print("Positive chats: " + str(len(positiveChats)))
print("Negative chats: " + str(len(negativeChats)))
labeledPositiveChats = []
labeledNegativeChats = []
for chat in positiveChats:
  labeledPositiveChats.append((chat, "positive"))
for chat in negativeChats:
  labeledNegativeChats.append((chat, "negative"))
allChats = labeledPositiveChats + labeledNegativeChats
shuffle(allChats)
wordIndex = {}
counter = {"i": 0}
def convert_string_to_numbers(chat):
  numberifiedChat = []
  for word in chat.strip().split():
    word = word.lower()
    punctuation = ""
    if word[-1] in string.punctuation:
      punctuation = word[-1]
      if punctuation not in wordIndex:
        wordIndex[punctuation] = [counter["i"], 1]
        counter["i"] = counter["i"] + 1
      else:
        wordIndex[punctuation][1] += 1
      word = word[:-1]
    if len(word) > 0:
      if word not in wordIndex:
        wordIndex[word] = [counter["i"], 1]
        counter["i"] = counter["i"] + 1
      else:
        wordIndex[word][1] += 1
      numberifiedChat.append(wordIndex[word][0])
      if punctuation != "":
        numberifiedChat.append(wordIndex[punctuation][0])
  return numberifiedChat

print(len(allChats))
#filter out duplicative chats
chatsConvertedToNumbers = []
for (chat, sentiment) in allChats:
  numberifiedChat = convert_string_to_numbers(chat)
  if(len(numberifiedChat) < 21):
    chatsConvertedToNumbers.append((numberifiedChat, sentiment))
wordCount  = len(wordIndex.keys())
print("word count: " + str(wordCount))
print("Cleaned up test sentences: " + str(len(chatsConvertedToNumbers)))
print(allChats[:10])

Positive chats: 1722
Negative chats: 2278
4000
word count: 2292
Cleaned up test sentences: 3958
[('Every experience, no matter how difficult, holds a lesson to be learned.', 'positive'), ("It's impossible to find a decent parking spot in this city.", 'negative'), (" You're not good enough to make it in this competitive industry.", 'negative'), (" The doctor's diagnosis was devastating and left me feeling hopeless.", 'negative'), ("I'm stuck in a rut with no way out.", 'negative'), (' You are deserving of love, kindness, and respect - from yourself and others.', 'positive'), (" I'm too old to make a career change now.", 'negative'), ('You are loved, you are worthy, and you are enough, just as you are.\n', 'positive'), (' Today is a brand new day full of exciting possibilities.', 'positive'), (' The team will lose the championship game due to their lack of skill.', 'negative')]


In [ ]:
# sort wordIndex by the second entry in each array in the 2d array
sortedWordIndex = sorted(wordIndex.items(), key=lambda x: x[1][1], reverse=True)
print(sortedWordIndex)
#substitute allChats first tuple entry with an array of numbers using the index value
for (chat, _) in allChats:
  for word in chat.strip().strip("\n").split():
    word = word.lower()


[('.', [10, 3936]), ('and', [8, 2097]), ('to', [38, 1643]), ('is', [29, 1487]), (',', [20, 1466]), ('the', [44, 1451]), ('a', [30, 1409]), ('you', [0, 1244]), ('your', [12, 872]), ('are', [1, 783]), ('of', [3, 724]), ('in', [18, 690]), ('my', [27, 577]), ('every', [34, 542]), ('new', [36, 404]), ('no', [76, 395]), ('be', [25, 392]), ('this', [19, 388]), ("i'm", [72, 362]), ('life', [28, 352]), ('never', [79, 323]), ('make', [41, 321]), ('i', [53, 296]), ('day', [35, 266]), ('for', [165, 266]), ('world', [21, 264]), ('me', [157, 241]), ('just', [164, 240]), ('can', [61, 236]), ('has', [153, 224]), ('will', [270, 224]), ('with', [6, 206]), ('not', [201, 205]), ("i'll", [78, 202]), ('start', [39, 200]), ('matter', [110, 196]), ('how', [111, 195]), ('opportunity', [37, 183]), ('it', [122, 176]), ('from', [197, 173]), ('today', [90, 169]), ('than', [257, 166]), ('always', [168, 164]), ('an', [57, 159]), ('good', [117, 158]), ('chance', [125, 158]), ('enough', [118, 156]), ('so', [22, 155]),

In [ ]:
# check if there are any repeats in allChats
for i in range(len(allChats)):
  for j in range(i+1, len(allChats)):
    if allChats[i][0] == allChats[j][0]:
      print(allChats[i])

I covert from From word-indexes to one-hot-vectors. The theory is each word has a unique meaning that is unknown to my model. This seemed better than the rather brittle embedding approaches I was seening in the literature.

In [ ]:
import numpy as np
chats_to_hot_one_formats = np.zeros(shape=(len(chatsConvertedToNumbers),20,wordCount+50), dtype="uint8")
count = 0
for i in chatsConvertedToNumbers:
  two_d_array = chats_to_hot_one_formats[count]
  count += 1
  word = 0
  for wordNumber in i[0]:
    two_d_array[word][wordNumber] = 1
    word += 1
print(chats_to_hot_one_formats)

def convert_string_to_hot_one(chat):
  hot_one_array = np.zeros(shape=(20,wordCount+50), dtype="uint8")
  word = 0

  numberified = convert_string_to_numbers(chat)
  for wordNumber in numberified:
    hot_one_array[word][wordNumber] = 1
    word += 1
  return hot_one_array

[[[1 0 0 ... 0 0 0]
  [0 0 1 ... 0 0 0]
  [0 1 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


Now for Defining the model.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the neural network model
class BinaryClassificationModel(nn.Module):
    def __init__(self):
        super(BinaryClassificationModel, self).__init__()
        self.layers = nn.ModuleList()

        # Input layer
        self.layers.append(nn.Linear((wordCount+50) * 20, 20))

        # Repeating dense layers
        for _ in range(1):
            self.layers.append(nn.Linear(20, 20))

        # Output layer
        self.output_layer = nn.Linear(20, 1)

    def forward(self, x):
        x = x.view(-1, (wordCount+50) * 20)  # Flatten the input
        for layer in self.layers:
            x = torch.relu(layer(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

# Instantiate the model
model = BinaryClassificationModel()

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Example function to train the model on one batch
def train_model(model, criterion, optimizer, test_data, test_labels, val_data, val_labels, epochs=20):
    for epoch in range(epochs):
        model.train()

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(test_data)
        loss = criterion(outputs, test_labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # check against validation set
        model.eval()
        with torch.no_grad():
            val_outputs = model(val_data)
            val_loss = criterion(val_outputs, val_labels)
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}')

Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x7994da4b8160>
Traceback (most recent call last):
  File "/usr/lib/python3.10/_weakrefset.py", line 39, in _remove
    def _remove(item, selfref=ref(self)):
KeyboardInterrupt: 


KeyboardInterrupt: 

Put labels in the correct format

In [ ]:
labelOfChats = []
for (chat, sentiment) in chatsConvertedToNumbers:
  if sentiment == "positive":
    labelOfChats.append([1])
  else:
    labelOfChats.append([0])
# Example labels (replace with your actual labels)
labels = torch.tensor(np.array(labelOfChats), dtype=torch.float32)  # Convert to a PyTorch tensor


Divide test and validation set, about 50/50 seemed good

In [ ]:

# Convert NumPy matrices to a PyTorch tensor
data_tensor = torch.tensor(chats_to_hot_one_formats, dtype=torch.float32)  # Convert to a PyTorch tensor
data_tensor = data_tensor.view(len(chats_to_hot_one_formats), -1)  # Reshape to (num_samples, 2500 * 20)

# split data_tensor to leave off the last 1000 elements
division = -1500
test_data = data_tensor[:division]
test_labels = labels[:division]
val_data = data_tensor[division:]
val_labels = labels[division:]
# Train the model
train_model(model, criterion, optimizer, test_data, test_labels, val_data, val_labels)

Epoch [1/20], Loss: 0.6902, Validation Loss: 0.6749
Epoch [2/20], Loss: 0.6718, Validation Loss: 0.6463
Epoch [3/20], Loss: 0.6390, Validation Loss: 0.6086
Epoch [4/20], Loss: 0.5963, Validation Loss: 0.5642
Epoch [5/20], Loss: 0.5450, Validation Loss: 0.5123
Epoch [6/20], Loss: 0.4852, Validation Loss: 0.4557
Epoch [7/20], Loss: 0.4210, Validation Loss: 0.3964
Epoch [8/20], Loss: 0.3558, Validation Loss: 0.3373
Epoch [9/20], Loss: 0.2931, Validation Loss: 0.2816
Epoch [10/20], Loss: 0.2364, Validation Loss: 0.2314
Epoch [11/20], Loss: 0.1870, Validation Loss: 0.1873
Epoch [12/20], Loss: 0.1448, Validation Loss: 0.1490
Epoch [13/20], Loss: 0.1088, Validation Loss: 0.1164
Epoch [14/20], Loss: 0.0787, Validation Loss: 0.0891
Epoch [15/20], Loss: 0.0544, Validation Loss: 0.0674
Epoch [16/20], Loss: 0.0358, Validation Loss: 0.0512
Epoch [17/20], Loss: 0.0226, Validation Loss: 0.0398
Epoch [18/20], Loss: 0.0139, Validation Loss: 0.0324
Epoch [19/20], Loss: 0.0083, Validation Loss: 0.0278
Ep

Now I can spot check values in the model


In [ ]:
# test the model against a new string
output = model(torch.tensor(convert_string_to_hot_one("You are mean."), dtype=torch.float32).view(1, -1))
output = output.detach().numpy()
if(output[0][0] > 0.5):
  print("positive")
else:
  print("negative")

positive
